# 연구목표

- 일반 쇼핑몰에서 발생한 데이터
  - 고객, 상품, 주문정보
  - 이를 이용하여 기본(초)적인 정량 분석 진행
- 목표
  - pandas를 좀더 편하게 사용함에 있다

# 데이터 수집

- 사내 데이터를 제공한다 (level 1)
- 내역
  - 고객 마스터 정보(1개)
  - 제품 마스터 정보(1개)
  - 주문 정보 (2개)
  - 주문 정보 상세 (2개)
  - 형식 csv로 제공
- 구글 드라이브에서 연결하여 사용

In [2]:
import os

src_path = '/content/drive/MyDrive/cloud_ai/share/2.데이터분석/res'

# 경로 + 파일 까지 풀경로를 자동으로 합쳐서 리턴처리
os.path.join( src_path, 'transaction_1.csv' ) 

'/content/drive/MyDrive/cloud_ai/share/2.데이터분석/res/transaction_1.csv'

# 데이터 준비


- transaction_1 + transaction_2 병합
- transaction_detail_1 + transaction_detail_2 병합
- customer_master, item_master 로드
- transaction_detail, transaction 병합(머지)
- 여기까지 해서 총 3개의 df가 준비된다
  - customer_master
  - item_master
  - transaction_join

In [4]:
import numpy as np
import pandas as pd

## csv 파일을 df로 로드한다

In [7]:
# 데이터 위치를 조정하여 자동으로 목록을 가져오게 처리 => 확장성을 확대
import glob

datas = glob.glob('/content/drive/MyDrive/cloud_ai/share/2.데이터분석/res/data/*.csv')
datas

['/content/drive/MyDrive/cloud_ai/share/2.데이터분석/res/data/customer_master.csv',
 '/content/drive/MyDrive/cloud_ai/share/2.데이터분석/res/data/item_master.csv',
 '/content/drive/MyDrive/cloud_ai/share/2.데이터분석/res/data/transaction_1.csv',
 '/content/drive/MyDrive/cloud_ai/share/2.데이터분석/res/data/transaction_2.csv',
 '/content/drive/MyDrive/cloud_ai/share/2.데이터분석/res/data/transaction_detail_1.csv',
 '/content/drive/MyDrive/cloud_ai/share/2.데이터분석/res/data/transaction_detail_2.csv']

In [8]:
# 같은 작업을 여러개 하니 이렇게 처리하겠다 -> 파일 순서는 기억해야 한다 --;;
dfs = [ pd.read_csv( data ) for data in datas ]

In [11]:
dfs[0].shape, dfs[-1].shape
# 순번 0:고객, 1:제품, 2-3:트렌젝션, 4-5:트렌젝션 디테일

((5000, 8), (2144, 4))

## 병합

-  transaction_1 + transaction_2 병합

In [12]:
# shape 확인
dfs[2].shape, dfs[3].shape

((5000, 4), (1786, 4))

In [13]:
dfs[2].head(2)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502
1,T0000000114,50000,2019-02-01 01:37:23,HD678019


In [14]:
dfs[3].head(2)

,transaction_id,price,payment_date,customer_id
0,T0000005113,295000,2019-06-15 07:20:27,TS169261
1,T0000005114,50000,2019-06-15 07:35:47,HI599892


- 2개의 df는 동일한 구조(컬럼), 차이는 오직 데이터량
- 병합 : concat(), axis = 0 : 수직으로 합병
- 인덱스가 : 0 ~ 4999, 0 ~ 21xx => 그냥둘것인가? 정리할것인가?(만들때 정리?, 만든후 정리?)

In [15]:
# 병합(단순 합치기, 수직방향)
transaction = pd.concat( [dfs[2], dfs[3]] )
transaction.shape

(6786, 4)

In [16]:
# 결합지점 데이터 확인 (4999~5001)
transaction.iloc[  5000-2:5000+2 , : ]
# 문제점 => 예상대로 인덷ㄱ스가 다시 처음부터 진행 -> 만들때 처리하는것으로 대체

,transaction_id,price,payment_date,customer_id
4998,T0000005111,210000,2019-06-15 04:14:06,IK074758
4999,T0000005112,50000,2019-06-15 04:42:38,HD444151
0,T0000005113,295000,2019-06-15 07:20:27,TS169261
1,T0000005114,50000,2019-06-15 07:35:47,HI599892


In [17]:
# ignore_index : 기존 인덱스 무시할것인가?
transaction = pd.concat( [dfs[2], dfs[3]],  ignore_index=True)
transaction.iloc[  5000-2:5000+2 , : ]

,transaction_id,price,payment_date,customer_id
4998,T0000005111,210000,2019-06-15 04:14:06,IK074758
4999,T0000005112,50000,2019-06-15 04:42:38,HD444151
5000,T0000005113,295000,2019-06-15 07:20:27,TS169261
5001,T0000005114,50000,2019-06-15 07:35:47,HI599892


In [18]:
transaction.tail(2)
# 마지막 데이터는 6785 인덱스값 체크 완료

,transaction_id,price,payment_date,customer_id
6784,T0000006897,85000,2019-07-31 23:39:35,TS624738
6785,T0000006898,85000,2019-07-31 23:41:38,AS834214


- transaction_detail_1 + transaction_detail_2 병합

In [19]:
transaction_detail =  pd.concat( [dfs[4], dfs[5]],  ignore_index=True)

display( transaction_detail )
transaction_detail.shape

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1
1,1,T0000000114,S001,1
2,2,T0000000115,S003,1
3,3,T0000000116,S005,1
4,4,T0000000117,S002,2
...,...,...,...,...
7139,7139,T0000006894,S004,1
7140,7140,T0000006895,S002,1
7141,7141,T0000006896,S001,2
7142,7142,T0000006897,S002,1


(7144, 4)

- transaction_detail, transaction 병합(머지)

In [20]:
transaction.head(1)

,transaction_id,price,payment_date,customer_id
0,T0000000113,210000,2019-02-01 01:36:57,PL563502


In [21]:
transaction_detail.head(1)

,detail_id,transaction_id,item_id,quantity
0,0,T0000000113,S005,1


- 대상이 **2개**의 df 사이에는 **transaction_id 라는 컬럼만 공통** 으로 존재 -> 병합 -> **merge**() 사용

- transaction에서 transaction_id	payment_date	customer_id 만 병합에 참가한다


In [25]:
# transaction df에서 transaction_id	payment_date	customer_id만 나오게 추출하시오
# 데이터를 추출하는 관점에서 획득
transaction[ ['transaction_id', 'payment_date',	'customer_id'] ].head(2)

,transaction_id,payment_date,customer_id
0,T0000000113,2019-02-01 01:36:57,PL563502
1,T0000000114,2019-02-01 01:37:23,HD678019


In [26]:
# 빠지는 관점에서 컬럼 1개만 제외하면 된다 -> drop
transaction.drop( ['price'], axis=1 ).head(1)

,transaction_id,payment_date,customer_id
0,T0000000113,2019-02-01 01:36:57,PL563502


In [27]:
# transaction에서 transaction_id	payment_date	customer_id 만 병합에 참가한다
transaction_join = pd.merge( transaction_detail, transaction.drop( ['price'], axis=1 ) )

transaction_join.shape

(7144, 6)

In [28]:
transaction_join.head(1)

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502


- transaction_join, 고객마스터( dfs[0] ) 합병

In [32]:
display( dfs[0].head(1) )
dfs[0].shape

,customer_id,customer_name,registration_date,email,gender,age,birth,pref
0,IK152942,김서준,2019-01-01 0:25,hirata_yuujirou@example.com,M,29,1990-06-10,대전광역시


(5000, 8)

In [31]:
# customer_id 가 중복된다
# transaction_join, dfs[0], left merge() 진행
# left기준이므로 트랜젝션 정보에 매칭되는 고객정보가 누락될수도 있다
tr_join_data = pd.merge( transaction_join, dfs[0], on='customer_id', how='left' )
tr_join_data.shape
# 총거래는 7144개 중에 2개이상 거래한 고객이 존재한다!!

(7144, 13)

In [33]:
tr_join_data.head(1)

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id,customer_name,registration_date,email,gender,age,birth,pref
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502,김태경,2019-01-07 14:34,imoto_yoshimasa@example.com,M,30,1989-07-15,대전광역시


- tr_join_data, 제품 데이터( dfs[1] ) 합병

In [34]:
dfs[1].head(1)

,item_id,item_name,item_price
0,S001,PC-A,50000


In [35]:
# item_id 컬럼 기준 머지
tr_join_data = pd.merge( tr_join_data, dfs[1], on='item_id', how='left' )
tr_join_data.shape
# 데이터 7144, 컬럼 15

(7144, 15)

In [36]:
tr_join_data.head(1)

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id,customer_name,registration_date,email,gender,age,birth,pref,item_name,item_price
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502,김태경,2019-01-07 14:34,imoto_yoshimasa@example.com,M,30,1989-07-15,대전광역시,PC-E,210000


- 검증
  - 수량*단가 = 주문당 매출(price)
  - 1. price 컬럼을 추가하시오 = 수량*단가
  - 2. price 컬럼과 트렌젝션 row데이터에 있던 price간의 총합이 동일한지 비교(간단하게 처리)
    - tr_join_data['price'].sum()

In [38]:
# tr_join_data라는 df에 price 컬럼을 추가하시오 = 수량*단가
# 파생변수의 값은 스칼라를 넣으면 동일값으로 세팅, 백터(Series)를 넣으면 각각 자리에 맞게 세팅
tr_join_data[ 'price' ] = tr_join_data['quantity'] * tr_join_data.item_price
# 'price' 컬럼값:종속변수 <= 'quantity' 컬럼과 'item_price' 컬럼에 영향을 받는다 : 독립변수
tr_join_data.head(1)

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id,customer_name,registration_date,email,gender,age,birth,pref,item_name,item_price,price
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502,김태경,2019-01-07 14:34,imoto_yoshimasa@example.com,M,30,1989-07-15,대전광역시,PC-E,210000,210000


In [41]:
# 합산해서 비교
tr_join_data.price.sum() == transaction.price.sum()
# 수량, 단가, 매출, 모두 일치한다(정상적인 데이터)

True

## 데이터 클리닝 작업

- 결측치 체크(간단하게 체크)

In [42]:
tr_join_data.info()
# 결측치가 없다

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7144 entries, 0 to 7143
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   detail_id          7144 non-null   int64 
 1   transaction_id     7144 non-null   object
 2   item_id            7144 non-null   object
 3   quantity           7144 non-null   int64 
 4   payment_date       7144 non-null   object
 5   customer_id        7144 non-null   object
 6   customer_name      7144 non-null   object
 7   registration_date  7144 non-null   object
 8   email              7144 non-null   object
 9   gender             7144 non-null   object
 10  age                7144 non-null   int64 
 11  birth              7144 non-null   object
 12  pref               7144 non-null   object
 13  item_name          7144 non-null   object
 14  item_price         7144 non-null   int64 
 15  price              7144 non-null   int64 
dtypes: int64(5), object(11)
memory usage: 948.

In [43]:
tr_join_data.isnull()
# 결측이 존재하면 어딘가에서 True가 세팅된다

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id,customer_name,registration_date,email,gender,age,birth,pref,item_name,item_price,price
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7139,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7140,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7141,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7142,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [44]:
tr_join_data.isnull().sum()
# False => 0, True => 1
# 한개라도 존재하면 컬럼별로 0 이상의 값이 될것이다

detail_id            0
transaction_id       0
item_id              0
quantity             0
payment_date         0
customer_id          0
customer_name        0
registration_date    0
email                0
gender               0
age                  0
birth                0
pref                 0
item_name            0
item_price           0
price                0
dtype: int64

In [45]:
# 기초 통계량
tr_join_data.describe()
# 숫자만 집계된다

,detail_id,quantity,age,item_price,price
count,7144.000000,7144.000000,7144.000000,7144.000000,7144.000000
mean,3571.500000,1.199888,50.265677,121698.628219,135937.150056
std,2062.439494,0.513647,17.190314,64571.311830,68511.453297
min,0.000000,1.000000,20.000000,50000.000000,50000.000000
25%,1785.750000,1.000000,36.000000,50000.000000,85000.000000
50%,3571.500000,1.000000,50.000000,102500.000000,120000.000000
75%,5357.250000,1.000000,65.000000,187500.000000,210000.000000
max,7143.000000,4.000000,80.000000,210000.000000,420000.000000


In [46]:
# 구매일 최소, 최대로 체크
# 이 데이터의 기간을 체크 가능
tr_join_data.payment_date.min(), tr_join_data.payment_date.max()
# 6개월치 데이터

('2019-02-01 01:36:57', '2019-07-31 23:41:38')

## 월별 데이터 집계

- 월별로 상품별로 매출 혹은 수량 집계하시오
  - shape : (6, 5)
- 인덱스 : 월별(payment_date -> 추출)
- 컬럼   : 제품명(item_name)
- 데이터 : 매출(price) 혹은 수량(주문서 개수가 아닌, 판매된 제품의 총수, quantity)

In [47]:
tr_join_data.head(1)

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id,customer_name,registration_date,email,gender,age,birth,pref,item_name,item_price,price
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502,김태경,2019-01-07 14:34,imoto_yoshimasa@example.com,M,30,1989-07-15,대전광역시,PC-E,210000,210000


In [48]:
# 제품은 총 5개 존재
tr_join_data.item_name.unique()

array(['PC-E', 'PC-A', 'PC-C', 'PC-B', 'PC-D'], dtype=object)

#### 월별 기준값 컬럼 생성

- 컬럼명 : pay_month
- 형식 : yyyyMM
- 재료 : payment_date
- 방식 : 문자열로 처리 or 시간/날짜 타입 처리

In [51]:
'2019-02-01 01:36:57'[:7].replace('-','')

'201902'

In [53]:
# 문자열 처리방식을 이용하여 추출
tr_join_data['pay_month'] = tr_join_data['payment_date'].apply( lambda x:x[:7].replace('-','') )
tr_join_data.head(1)

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id,customer_name,registration_date,email,gender,age,birth,pref,item_name,item_price,price,pay_month
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502,김태경,2019-01-07 14:34,imoto_yoshimasa@example.com,M,30,1989-07-15,대전광역시,PC-E,210000,210000,201902


In [55]:
# 시간/날짜 타입으로 처리
# 타입체크
# tr_join_data.payment_date.dtype
# payment_date         object => 문자열일 가능성이 높다
tr_join_data.dtypes
# 데이터는 시간/날짜 포멧을따른다 => 시간형 타입으로 변환 처리

detail_id             int64
transaction_id       object
item_id              object
quantity              int64
payment_date         object
customer_id          object
customer_name        object
registration_date    object
email                object
gender               object
age                   int64
birth                object
pref                 object
item_name            object
item_price            int64
price                 int64
pay_month            object
dtype: object

In [56]:
# 1, 타입을 datetime 형으로 변환
tr_join_data['payment_date'] = pd.to_datetime( tr_join_data['payment_date'] )
tr_join_data.dtypes

detail_id                     int64
transaction_id               object
item_id                      object
quantity                      int64
payment_date         datetime64[ns]
customer_id                  object
customer_name                object
registration_date            object
email                        object
gender                       object
age                           int64
birth                        object
pref                         object
item_name                    object
item_price                    int64
price                         int64
pay_month                    object
dtype: object

In [57]:
# 2. 데이터를 추출해서 시간 포멧팅을 활용하여 추출
# datetime형.dt.strftime('시간포멧')
tr_join_data['pay_month'] = tr_join_data['payment_date'].dt.strftime('%Y%m')

In [58]:
tr_join_data.head(1)

,detail_id,transaction_id,item_id,quantity,payment_date,customer_id,customer_name,registration_date,email,gender,age,birth,pref,item_name,item_price,price,pay_month
0,0,T0000000113,S005,1,2019-02-01 01:36:57,PL563502,김태경,2019-01-07 14:34,imoto_yoshimasa@example.com,M,30,1989-07-15,대전광역시,PC-E,210000,210000,201902


#### pay_month(월별) 기준으로 집계

In [59]:
tr_join_data.pay_month.unique()
# 집계의 기준이 되는 컬럼의 데이터는 범주형으로 확인 되었고, 
# 각각의 값을 기준으로 그룹화 처리 가능 -> 집계가능

array(['201902', '201903', '201904', '201905', '201906', '201907'],
      dtype=object)

In [66]:
tmp = tr_join_data.groupby( ['pay_month', 'item_name']).sum()[ ['price'] ]
tmp.shape # (30, 1) -> ( 월별(6),제품별(5) )로 재구성하시오

(30, 1)

- 피벗을 통해서 의도한 df를 구성

In [70]:
# 원본에서 작업
tmp = pd.pivot_table( tr_join_data,  index='pay_month', 
                columns='item_name', values='price', aggfunc='sum' )
tmp.shape, display(tmp)

item_name,PC-A,PC-B,PC-C,PC-D,PC-E
pay_month,,,,,
201902,24150000,25245000,19800000,31140000,59850000
201903,26000000,25500000,19080000,25740000,64050000
201904,25900000,23460000,21960000,24300000,64890000
201905,24850000,25330000,20520000,25920000,58800000
201906,26000000,23970000,21840000,28800000,63420000
201907,25250000,28220000,19440000,26100000,71610000


((6, 5), None)

# 데이터 분석

# 모델 구축(생략)

# 시스템 통합(생략)